# 数据合成 baseline

## 基础工具函数

In [1]:
import json
import re
import requests

# def llm(content):
#     if isinstance(content, str):
#         messages = [{'role': 'user', 'content': content}]
#     elif isinstance(content, list):
#         messages = content
#     else:
#         raise ValueError('输入格式错误')
#     data = {
#         "messages":messages,
#         "max_tokens": 8000,
#         "temperature": 0.1,
#         "response_format": {"type": "json_object"}
#     }
#     headers = {"Content-Type": "application/json",
#                "Authorization": "Bearer XXX"}
#
#     res = requests.post("https://newqingjing.com/competition/apitest", json=data, headers=headers).json()
#     answer = res['choices'][0]['message']['content']
#     if isinstance(content, list):
#         content.append({"role": "assistant", "content": answer})
#     return answer

# from llm import llmds as llm

def get_json(content):
    print(content)
    json_str = re.findall('```json(.*?)```',content,re.DOTALL)

    return json.loads(json_str[-1])

## 合成最初版数据，每次保存前几次合成的数据，可以多样化

In [ ]:

prompt = '''
我有一批测试集，用于检测大模型性能，测试大模型性能主要测试其指令遵循和复杂格式生成能力，以下是单条示例：
-------测试集1-----
```json
{
  "id": 0,
  "messages": [
    {
      "role": "user",
      "content": "请你随机产生50个生活中常用的问题，例如，包含，医学，法律，生物学等知识。按照以下json格式输出:
```json
[{"id":1,"content":"问题1",...,{"id":50,"content":"问题50"}]
```,结果在```json ```内"
    }
  ],
  "condition": [
    {
      "msg_index": 1,
      "constraint_type": "个数约束",
      "constraint_detail": "必须严格保证50个数量，ID顺序正确，医学，法律，生物学知识分别不少于十条，必须有其他学科知识"
    },
    {
      "msg_index": 2,
      "constraint_type": "文本约束",
      "constraint_detail": "必须以```json开始并且以```结束，json格式必须正确，字段名称必须正确，仔细检查有没有格式错乱，中文标点等错误"
    }
  ]
}
```
-----------------------
---------测试集2--------
```json
{
  "id": 0,
  "messages": [
    {
      "role": "user",
      "content": "你是一位营养学领域资深的营养师，特别擅长创作食谱。
## 任务描述
食谱创作：根据系统规范和食谱名称、烹饪工具、烹饪功能，创作健康、美味、可以安全食用的食谱。
## Goals
1. 按照JSON数组格式输出，该JSON数组可以被解析，因此不能够有任何注释内容
2. 根据用户特定要求（如电饭煲、煲汤等）创作
3. 提升食谱的质量
4. 食谱创作不要编造完全不存在的食谱
## Skills
1. 你是一位顶级的营养师、美食达人，擅长创作食谱
2. 熟练掌握食谱的创作技巧
3. 理解并能创作多种菜系（如：粤菜、川菜、鲁菜、本帮菜等）的食谱
4. 能够基于特定的食谱结构要求完成创作
5. 对用户需求保持敏感，及时调整创作方向
## OutputFormat
- 输出形式：python object格式数组，其中一个为：，
- 示例格式：
```python
{
  'recipeName': '虾仁豆腐煲仔饭',
  'mainMaterials': [{'name': '大米', 'value': 150, 'unit': 'g'}],
  'ingredient': [{'name': '青豆', 'value': 50, 'unit': 'g'}],
  'condiment': [{'name': '生抽', 'value': 15, 'unit': 'g'}],
  'cleverWays': {
    'ingredientsHandle': '虾仁去壳去肠泥，用盐和白胡椒粉腌制10分钟。嫩豆腐切块，香菇切片，胡萝卜切丁。',
    'attention': '使用苏泊尔电饭煲SF30HC85A的煲仔饭功能时，确保所有食材均匀铺在米饭上，以便受热均匀。'
  },
  'step': [
    '1. 大米洗净后，加入电饭煲内胆，加入适量水（水量根据电饭煲指示或个人口味调整）。',
    '2. 在大米上均匀铺上腌制好的虾仁、嫩豆腐块、青豆、胡萝卜丁和香菇片。',
    '3. 将生抽、蚝油、盐和香油混合均匀后，淋在食材上。',
    '4. 盖上电饭煲盖，选择煲仔饭功能，启动电饭煲。',
    '5. 电饭煲工作完成后，让饭焖5分钟再开盖，搅拌均匀即可享用美味的虾仁豆腐煲仔饭。'
  ]
}```,请分别输出粤菜、川菜、鲁菜的做法，为list格式"
    }
  ],
  "condition": [
    {
      "msg_index": 1,
      "constraint_type": "个数约束",
      "constraint_detail": "必须输出粤菜，鲁菜，川菜的做法，一种三种，格式为 list[object]"
    },
    {
      "msg_index": 2,
      "constraint_type": "文本约束",
      "constraint_detail": "必须以```python开始并且以```结束，数据格式必须正确，仔细检查有没有格式错乱，字段名称必须正确，中文标点等错误"
    }
  ]
}
```
--------------------
其中约束类型可以为:
📊 约束类型分类
🎨 形式约束
体裁约束：规定输出文本的文体类型（诗歌、散文、记叙文等）
格式约束：要求输出遵循特定数据格式或编程语言格式
排版约束：控制文本视觉呈现方式（字体、缩进、表格等）
🔢 数量约束
个数约束：限制输出中某些元素的数量范围
长度约束：控制输出文本长度（字数、句数、段数等）
🗣️ 语言约束
中文约束：要求使用中文（简体或繁体）进行输出
英文约束：要求使用英文进行输出
其他语言约束：要求使用除中英文外的其他指定语言
💭 语义约束
示例约束：通过具体示例引导模型输出格式和内容
风格技巧约束：规定输出的语言风格、写作技巧和表达方式
情感倾向约束：控制输出内容的情感色彩（正向、中立、负向）
📝 内容约束
原文约束：规定输出与原文的关系（必须引用或不能复制）
符号约束：要求输出中包含或不包含特定标点符号表情符号
词汇约束：规定输出中必须使用或禁止使用的特定词汇
集合约束：通过列表形式限定某字段的可选取值范围
数值约束：对输出中数值字段设置取值范围限制
文本约束：通过文本描述规定输出中应包含的具体内容
时间约束：要求输出内容符合特定时间维度要求
主题约束：限定输出内容必须围绕指定主题进行
🏗️ 结构流程约束
结构约束：规定输出的组织结构（分几部分、有无标题等）
流程约束：控制输出内容的先后顺序和执行流程
边界约束：定义系统执行任务时的能力边界和行为规范
其他约束：不属于上述分类的复杂或非典型约束要求
'''

messages = [{"role": "system", "content": prompt},
            {"role":"user",'content':"请生成一个测试集 必须包含  流程约束 和  排版约束 并使用复杂输出结构,仅仅输出结果"},
            {"role":"assistant",'content':"""{
  "id": 1,
  "messages": [
    {
      "role": "user",
      "content": "请生成一个软件开发生命周期的详细计划。生命周期包括以下阶段：需求分析、系统设计、编码、测试、部署。每个阶段必须包含以下信息：阶段名称、描述、输入文档、输出文档、负责人、子任务列表。子任务列表必须包括至少三个子任务，每个子任务有名称、描述、预计工时。输出必须按照生命周期的顺序排列阶段，即需求分析第一，部署最后。输出格式为JSON，结构如下：\n```json\n{\n  \"projectName\": \"示例项目\",\n  \"phases\": [\n    {\n      \"phaseName\": \"需求分析\",\n      \"description\": \"阶段描述\",\n      \"inputs\": [\"输入文档1\", \"输入文档2\"],\n      \"outputs\": [\"输出文档1\", \"输出文档2\"],\n      \"responsible\": \"负责人角色\",\n      \"subTasks\": [\n        {\n          \"taskName\": \"子任务名称\",\n          \"description\": \"子任务详细描述\",\n          \"estimatedHours\": 8\n        }\n      ]\n    }\n  ]\n}\n```\n输出时，将整个JSON放在```json块中，并使用漂亮的打印（缩进2个空格），确保字段名使用双引号，字符串使用双引号。"
    }
  ],
  "condition": [
    {
      "msg_index": 1,
      "constraint_type": "流程约束",
      "constraint_detail": "阶段必须严格按照顺序输出：需求分析 → 系统设计 → 编码 → 测试 → 部署，不能颠倒或缺失任何阶段。"
    },
    {
      "msg_index": 2,
      "constraint_type": "排版约束",
      "constraint_detail": "JSON输出必须以```json开始并以```结束，使用2个空格的缩进，字段名和字符串必须使用双引号，不得使用单引号或无引号。整体格式必须美观，无缩进错误或中文标点。"
    }
  ]
}"""}]

results = []

n=1
all_yueshu = [ "体裁约束", "格式约束", "排版约束", "个数约束", "长度约束", "中文约束", "英文约束", "其他语言约束", "示例约束", "风格技巧约束", "情感倾向约束", "原文约束", "符号约束", "词汇约束", "集合约束", "数值约束", "文本约束", "时间约束", "主题约束", "结构约束", "流程约束", "边界约束", "其他约束"]
import random
for i in range(50):
    # 随机选两个约束
    a,b = random.sample(all_yueshu,2)

    messages.append({"role":"user",'content':f"请生成一个测试集 必须包含  {a} 和  {b} 并使用复杂输出结构,仅仅输出结果"})
    if len(messages)>10:
        messages.pop(1)
        messages.pop(1)

    answer= llm(messages)
    try:
        data = json.loads(answer)
        data['id'] = n
        n += 1
        with open('tmp.jsonl', 'a', encoding='utf8') as fw:
            fw.write(json.dumps(data, ensure_ascii=False))
            fw.write('\n')

    except:
        import traceback
        traceback.print_exc()


## 在原有基础上优化合成的数据

In [ ]:
prompt = '''
我有一批测试集，用于检测大模型性能，测试大模型性能主要测试其指令遵循和复杂格式生成能力，以下是单条示例：
-------测试集1-----
```json
{
  "id": 0,
  "messages": [
    {
      "role": "user",
      "content": "请你随机产生50个生活中常用的问题，例如，包含，医学，法律，生物学等知识。按照以下json格式输出:
```json
[{"id":1,"content":"问题1",...,{"id":50,"content":"问题50"}]
```,结果在```json ```内"
    }
  ],
  "condition": [
    {
      "msg_index": 1,
      "constraint_type": "个数约束",
      "constraint_detail": "必须严格保证50个数量，ID顺序正确，医学，法律，生物学知识分别不少于十条，必须有其他学科知识"
    },
    {
      "msg_index": 2,
      "constraint_type": "文本约束",
      "constraint_detail": "必须以```json开始并且以```结束，json格式必须正确，字段名称必须正确，仔细检查有没有格式错乱，中文标点等错误"
    }
  ]
}
```
-----------------------
---------测试集2--------
```json
{
  "id": 0,
  "messages": [
    {
      "role": "user",
      "content": "你是一位营养学领域资深的营养师，特别擅长创作食谱。
## 任务描述
食谱创作：根据系统规范和食谱名称、烹饪工具、烹饪功能，创作健康、美味、可以安全食用的食谱。
## Goals
1. 按照JSON数组格式输出，该JSON数组可以被解析，因此不能够有任何注释内容
2. 根据用户特定要求（如电饭煲、煲汤等）创作
3. 提升食谱的质量
4. 食谱创作不要编造完全不存在的食谱
## Skills
1. 你是一位顶级的营养师、美食达人，擅长创作食谱
2. 熟练掌握食谱的创作技巧
3. 理解并能创作多种菜系（如：粤菜、川菜、鲁菜、本帮菜等）的食谱
4. 能够基于特定的食谱结构要求完成创作
5. 对用户需求保持敏感，及时调整创作方向
## OutputFormat
- 输出形式：python object格式数组，其中一个为：，
- 示例格式：
```python
{
  'recipeName': '虾仁豆腐煲仔饭',
  'mainMaterials': [{'name': '大米', 'value': 150, 'unit': 'g'}],
  'ingredient': [{'name': '青豆', 'value': 50, 'unit': 'g'}],
  'condiment': [{'name': '生抽', 'value': 15, 'unit': 'g'}],
  'cleverWays': {
    'ingredientsHandle': '虾仁去壳去肠泥，用盐和白胡椒粉腌制10分钟。嫩豆腐切块，香菇切片，胡萝卜切丁。',
    'attention': '使用苏泊尔电饭煲SF30HC85A的煲仔饭功能时，确保所有食材均匀铺在米饭上，以便受热均匀。'
  },
  'step': [
    '1. 大米洗净后，加入电饭煲内胆，加入适量水（水量根据电饭煲指示或个人口味调整）。',
    '2. 在大米上均匀铺上腌制好的虾仁、嫩豆腐块、青豆、胡萝卜丁和香菇片。',
    '3. 将生抽、蚝油、盐和香油混合均匀后，淋在食材上。',
    '4. 盖上电饭煲盖，选择煲仔饭功能，启动电饭煲。',
    '5. 电饭煲工作完成后，让饭焖5分钟再开盖，搅拌均匀即可享用美味的虾仁豆腐煲仔饭。'
  ]
}```,请分别输出粤菜、川菜、鲁菜的做法，为list格式"
    }
  ],
  "condition": [
    {
      "msg_index": 1,
      "constraint_type": "个数约束",
      "constraint_detail": "必须输出粤菜，鲁菜，川菜的做法，一种三种，格式为 list[object]"
    },
    {
      "msg_index": 2,
      "constraint_type": "文本约束",
      "constraint_detail": "必须以```python开始并且以```结束，数据格式必须正确，仔细检查有没有格式错乱，字段名称必须正确，中文标点等错误"
    }
  ]
}
```
--------------------
其中约束类型可以为:
📊 约束类型分类
🎨 形式约束
体裁约束：规定输出文本的文体类型（诗歌、散文、记叙文等）
格式约束：要求输出遵循特定数据格式或编程语言格式
排版约束：控制文本视觉呈现方式（字体、缩进、表格等）
🔢 数量约束
个数约束：限制输出中某些元素的数量范围
长度约束：控制输出文本长度（字数、句数、段数等）
🗣️ 语言约束
中文约束：要求使用中文（简体或繁体）进行输出
英文约束：要求使用英文进行输出
其他语言约束：要求使用除中英文外的其他指定语言
💭 语义约束
示例约束：通过具体示例引导模型输出格式和内容
风格技巧约束：规定输出的语言风格、写作技巧和表达方式
情感倾向约束：控制输出内容的情感色彩（正向、中立、负向）
📝 内容约束
原文约束：规定输出与原文的关系（必须引用或不能复制）
符号约束：要求输出中包含或不包含特定标点符号表情符号
词汇约束：规定输出中必须使用或禁止使用的特定词汇
集合约束：通过列表形式限定某字段的可选取值范围
数值约束：对输出中数值字段设置取值范围限制
文本约束：通过文本描述规定输出中应包含的具体内容
时间约束：要求输出内容符合特定时间维度要求
主题约束：限定输出内容必须围绕指定主题进行
🏗️ 结构流程约束
结构约束：规定输出的组织结构（分几部分、有无标题等）
流程约束：控制输出内容的先后顺序和执行流程
边界约束：定义系统执行任务时的能力边界和行为规范
其他约束：不属于上述分类的复杂或非典型约束要求
'''

import json
from concurrent.futures import ThreadPoolExecutor


def process_line(i):
    try:
        messages = [{"role": "system", "content": prompt},
                    {"role": "user", 'content': "请生成一个测试集,仅仅输出结果"},
                    {"role": "assistant", 'content': i}]

        for _ in range(3):
            messages.append({"role": "user", "content": """
    优化上面的测试集，其中 messages -> content 字段必须严格约束大模型输出，使用更多的约束条件，content应该详细而具体。
    并且上面的测试集修改的更加复杂，修改方法为：
    1. 输出更多更长的json内容，如输出3个->10个，2. 更复杂的json结构
    3. 增加更多的约束条件，例如不能出现xx字，4. 更严格的判分条件，想办法使得每个判分条件更加严格 5. messages,必要时改为多轮对话，
    messages ->[{"role": "user", 'content': "XXX"},{"role": "assistant", 'content': "XXX"},{"role": "user", 'content': "XXX"}]
    但是最后一个元素的role必须是user
    仅仅输出结果,判分。
            """})

            answer = llm(messages)
            data = json.loads(answer)

        return data
    except:
        return {}

with open('tmp.jsonl', encoding='utf8') as f, \
        open('submit.jsonl', 'a', encoding='utf8') as fw, \
        ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for i in f:
        future = executor.submit(process_line, i)
        futures.append(future)

    for future in futures:
        data = future.result()
        fw.write(json.dumps(data, ensure_ascii=False) + '\n')